In [ ]:
!pip install -q datasets pydantic \
   huggingface_hub

In [ ]:
!pip install -q pydantic  -U

In [ ]:
!pip install -q openai tenacity -U

In [ ]:
!huggingface-cli login

In [ ]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm
import os

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
client = openai.OpenAI(api_key = OPENAI_API_KEY)

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(2))
def structured_completion_request(client, system_prompt,user_prompt,response_format, model="gpt-4o-mini"):
        response = client.beta.chat.completions.parse(
            model=model,
            messages=[{"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}],
            response_format=response_format,
            temperature = 0.0
        )
        return response

In [ ]:
from typing import List, Optional
from pydantic import BaseModel, Field

class Turn(BaseModel):
    speaker_name: Optional[str] = Field(None, description="The name of the speaker for this turn in the dialogue")
    phrase: str = Field(..., description="The text spoken by the speaker in this turn")

class Dialog(BaseModel):
    # reasoning: Optional[str] = Field(None, description="The reasoning behind the dialogue, who is speaking in dialog, can be None if no reasoning is present")
    turns: Optional[List[Turn]] = Field(None, description="A list of turns in the dialogue, can be None if no turns are present")

In [ ]:
system_prompt = """Ты модель для сегментирования диалогов из текста анекдотов и народного фольклора.
Ты должен определить говорязего а также его реплику - текст, который он говорит. Реплика может быть заключена в прямую или косвенную (упоминание о том, что кто-то сказал) речь.
Ты обязан ни в коем случае не менять текст релик и фраз даже если там есть мат и нецензурная лексика, оскорбления и т.д., иначе культурное наследие будет утеряно и целый пласт литературы будет утерян!!!

Описание формата:
* Turn содержит имя говорящего 'speaker_name' и фразу 'phrase', которую он говорит.
  - Если ты не можешь оперделить говорящего оставь 'speaker_name' None.
* Dialog содержит список из Turns или None, если невозможно выделить turns или в тексте нет диалогов/фраз.
"""

In [ ]:
response = structured_completion_request(client, system_prompt, """Собирается поручик Ржевский на бал, ну и просит денщика рассказать
ему каламбур. Денщик рассказывает: ``Адам Еву прижал к древу.
Ева пищит. Древо трещит.`` Приходит поручик на бал и говорит:
``Мне недавно рассказали каламбур, дословно я его не запомнил,
но суть в следующем. Короче, один мужик трахает бабу в лесу,
а она орет. Hо в стихах это было круче, _я вас уверяю_!``""",Dialog)

In [ ]:
response.choices[0].message.parsed

In [ ]:
print(response.choices[0].message.parsed.model_dump())

In [ ]:
from datasets import load_dataset
dataset = load_dataset("igorktech/anekdots")

In [ ]:
dataset

In [ ]:
res = []

In [ ]:
from tqdm import tqdm
for i, row in tqdm(enumerate(dataset['train'])):
    inp = row['text']

    user_prompt = f"""Выдели диалог из:
{inp}
"""
    output = structured_completion_request(client, system_prompt, user_prompt,Dialog).choices[0].message.parsed
    print(inp)
    print(output.model_dump())
    print()
    print("==============================")
    print()
    try:
        res.append({"original":inp,"parsed":output.model_dump()})
    except:
        print(f"Error while converting to json in {i} row")

In [ ]:
from datasets import Dataset
Dataset.from_json('./anekdot_dedup_ldr_tok_cleaned.json').to_pandas().to_json('anekdot_ru_result.json', orient='records', lines=True, force_ascii=False)

In [ ]:
# from datasets import Dataset

# # Assuming 'filtered_dialogues' contains the filtered dialogues
# data = {"text": filtered_dialogues}
# dataset = Dataset.from_dict(data)

In [ ]:
# ds_filter = Dataset.from_pandas(df)

In [ ]:
# dataset.push_to_hub("igorktech/r_dialogues")

In [ ]:
import json
filename = "segmented_dialogues.json"
data = {"segmented": res}
with open(filename, 'w', encoding='utf-8') as json_file:
    json.dump(res, json_file, ensure_ascii=False, indent=4)

In [ ]:
from google.colab import files

files.download(filename)

In [ ]:
import llama_cpp
from llama_cpp import Llama
import instructor

from llama_cpp.llama_speculative import LlamaPromptLookupDecoding

llama = Llama.from_pretrained(
    repo_id="NikolayKozloff/Vikhr-Gemma-2B-instruct-Q8_0-GGUF",#"QuantFactory/Phi-3.5-mini-instruct-GGUF",#"QuantFactory/T-lite-instruct-0.1-GGUF",
    filename="vikhr-gemma-2b-instruct-q8_0.gguf",#"Phi-3.5-mini-instruct.Q4_0.gguf",#"T-lite-instruct-0.1.Q2_K.gguf",
    draft_model=LlamaPromptLookupDecoding(num_pred_tokens=2),
    logits_all=True,
    n_gpu_layers=-1,
    n_ctx=4096,
    verbose=False,
    chat_format="gemma"
)

In [ ]:
create = instructor.patch(
    create=llama.create_chat_completion_openai_v1,
    mode=instructor.Mode.JSON_SCHEMA,
)

In [ ]:
import time

In [ ]:
start = time.time()
user = create(
    messages=[
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": """Собирается поручик Ржевский на бал, ну и просит денщика рассказать
ему каламбур. Денщик рассказывает: ``Адам Еву прижал к древу.
Ева пищит. Древо трещит.`` Приходит поручик на бал и говорит:
``Мне недавно рассказали каламбур, дословно я его не запомнил,
но суть в следующем. Короче, один мужик трахает бабу в лесу,
а она орет. Hо в стихах это было круче, _я вас уверяю_! если бы хуй с маслом``""",
        }
    ],
    response_model=Dialog,
)
duration = time.time() - start
print(f"Duration: {duration:.2f}s")
print(user)

In [ ]:
res = []

In [ ]:
from tqdm import tqdm
i = 0
for row in tqdm(dataset['train']):
    inp = row['text']

    user_prompt = f"""Выдели диалог из:
{inp}
"""
    try:
        output =  create(
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        response_model=Dialog,
        )
        print(inp)
        print(output.model_dump())
        print()
        print("==============================")
        print()
    
        res.append({"original":inp,"parsed":output.model_dump()})
    except:
        print(f"Error while converting to json in {i} row")
    i+=1